# Julia's Type System

https://docs.julialang.org/en/v1/manual/types/


Two different type systems in programming languages. Static type systems, where every program expression must have a type computable before the execution of the program, and dynamic type systems, where nothing is known about types until run time, when the actual values manipulated by the program are available.

The ability to write code that can operate on different types is called polymorphism.

Julia's type system is dynamic, but gains some of the advantages of static type systems by making it possible to indicate that certain values are of specific types. This can be of great assistance in generating efficient code, but even more significantly, it allows method dispatch on the types of function arguments to be deeply integrated with the language. Method dispatch is explored in detail in the Multiple Dispatch notebooks, but is rooted in the type system presented here.



## Part 0. Type Declarations

The :: operator can be used to attach type annotations to expressions and variables in programs. There are two primary reasons to do this:

    1. As an assertion to help confirm that your program works the way you expect,
    2. To provide extra type information to the compiler, which can then improve performance in some cases

In [1]:
(1+2)::AbstractFloat

TypeError: TypeError: in typeassert, expected AbstractFloat, got a value of type Int64

In [2]:
(1+2)::Int

3

### Composite types
Composite types are introduced with the `struct` keyword followed by a block of field names, optionally annotated with types using the `::` operator:

In [3]:
struct Person
    name::String
    age::Int
    occupation
end

Fields with no type annotation default to `Any`, and can accordingly hold any type of value.

In [4]:
yiannis = Person("Yiannis",27,"PhD Student")

Person("Yiannis", 27, "PhD Student")

In [5]:
typeof(yiannis)

Person

In [6]:
yiannis2 = Person("Yiannis",27.5,"PhD Student")

InexactError: InexactError: Int64(27.5)

You may find a list of field names using the `fieldnames` function.

In [7]:
fieldnames(Person)

(:name, :age, :occupation)

You can access the field values of a composite object using the traditional `foo.bar` notation:

In [8]:
yiannis.age

27

Composite objects declared with `struct` are immutable; they cannot be modified after construction. This may seem odd at first, but it has several advantages:

* It can be more efficient. Some structs can be packed efficiently into arrays, and in some cases the compiler is able to avoid allocating immutable objects entirely.
* It is not possible to violate the invariants provided by the type's constructors.
* Code using immutable objects can be easier to reason about.

In [9]:
# Redefinition on immutable struct is invalid
struct Person
    name::String
    age::Int
    occupation
    favourite_food::String
end

ErrorException: invalid redefinition of constant Person

If a composite type is declared with `mutable struct` instead of `struct`, then instances of it can be modified:

In [10]:
mutable struct Improved_person
    name::String
    age::Int
    occupation
    favourite_food::String
end

### Abstract vs. Concrete

`abstract` types can have declared subtypes, while concrete types can have instances. These are separated because if an `X` IS-A `Y`, and `Y` specifies a representation, then `X` had better have the same representation.

`abstract` types cannot be instantiated and serve only as nodes in the type graph.

"car is-a vehicle" is correct because "vehicle" is an abstract concept that doesn't commit to any specifics. But if I tell you I'm giving you a Porsche, it needs to look like a Porsche.

A type `T` is concrete if there could be some value `x` such that `typeof(x) === T`. This is also sometimes called a "leaf type".

Abstract types are declared using the `abstract type` keyword. The general syntaxes for declaring an abstract type are:

In [11]:
abstract type name end # name is the name of the type...


Let's consider some of the abstract types that make up Julia's numerical hierarchy:

```julia
abstract type Number end
abstract type Real     <: Number end
abstract type AbstractFloat <: Real end
abstract type Integer  <: Real end
abstract type Signed   <: Integer end
abstract type Unsigned <: Integer end
```

The `<:` operator in general means "is a subtype of", and, used in declarations like this, declares the right-hand type to be an immediate supertype of the newly declared type. It can also be used in expressions as a subtype operator which returns `true` when its left operand is a subtype of its right operand:

In [12]:
Integer <: Number


true

In [13]:
Integer <: AbstractFloat

false

In [14]:
Complex <: Number

true

## Part 1. DataType
Julia programs manipulate *values*, and every value has two parts: a *type* part and a data part. The type part answers the question "what kind of thing is this?", and the data part distinguishes one thing of a certain kind from every other thing of that kind.


In [15]:
typeof(3)

Int64

In this case the type is `Int64` and the data part is the bits `...0011`.


In Julia types are also values:

In [16]:
typeof(Int64)

DataType

In [17]:
typeof(DataType)

DataType

In fact, the identity `typeof(typeof(x)) === DataType` holds for all values in Julia. `DataType` is the backbone of the entire system. It does many jobs, which can be identified by looking inside a `DataType` object:

### DataType Job 1: A symbolic description
This consists of a name (which is mostly a string), and a vector of sub-components:

In [18]:
T = typeof(1+2im)

Complex{Int64}

In [19]:
T.name

typename(Complex)

In [20]:
T.parameters

svec(Int64)

### DataType Job 2: A nominal hierarchy of types
DataTypes form a tree of declared type relationships ("an x is-a y"). Recall `Int <: Number`

In [22]:
T.super

Number

In [23]:
T.super.super.super.super  # `Any` is the built-in top of the hierarchy.

Any

### DataType Job 3: Describe the representation

In [24]:
T.types

svec(Int64, Int64)

In [25]:
T2 = typeof(1.5+2im)

ComplexF64 (alias for Complex{Float64})

In [26]:
T.name.names

svec(:re, :im)

In [27]:
T.size

16

In [28]:
T.mutable   # whether this was declared with `type` (vs. `immutable`)

ErrorException: type DataType has no field mutable

In [29]:
T.abstract  # whether this was declared with `abstract`

ErrorException: type DataType has no field abstract

In [30]:
T.ninitialized

ErrorException: type DataType has no field ninitialized

In [31]:
T.layout

Ptr{Nothing} @0x00007fcc503f3c20

## Part 2. Type parameters


Type parameters can be completely or partially specified:

In [32]:
Array{Int,1}

Vector{Int64} (alias for Array{Int64, 1})

In [33]:
[1] isa Array

true

In [34]:
Array{Int,2}

Matrix{Int64} (alias for Array{Int64, 2})

A type is concrete (can have instances) if
    1. it is not declared `abstract`
    2. all parameters are specified

In [35]:
[1] isa Array{Int,1}

true

In [36]:
[1] isa Array{Int}

true

In [37]:
[1] isa Array{Number}

false

In [38]:
Int <: Number

true

### Defining types with parameters

In [39]:
struct GenericPoint{T<:Real}
    x::T
    y::T
end

In [40]:
GenericPoint(1,2)

GenericPoint{Int64}(1, 2)

In [41]:
GenericPoint(1.0,2.0)

GenericPoint{Float64}(1.0, 2.0)

In [42]:
GenericPoint(1,2.0)

MethodError: MethodError: no method matching GenericPoint(::Int64, ::Float64)
Closest candidates are:
  GenericPoint(::T, !Matched::T) where T<:Real at ~/Repositories/Introduction-to-Julia/day2/08. Types.ipynb:2

### Tuple types

In [43]:
typeof((1,2.0))

Tuple{Int64, Float64}

Very similar to other DataTypes, except
    1. Have no field names, only indices
    2. `T.parameters == T.types`
    3. Are always immutable
    4. Can have any number of fields

These factors conspire to make Tuples the only *covariant* types in Julia:

In [44]:
Tuple{Int} <: Tuple{Number}

true

A Tuple type is concrete iff all its field types are.

Tuple types can be abstract with respect to the number of elements. These are called variadic tuple types, or vararg types.

In [45]:
Tuple{Int,Vararg{Int}}

Tuple{Int64, Vararg{Int64}}

Note that `Vararg` refers to the tail of a tuple type, and as such is not a first-class type itself. It only makes sense inside a Tuple type. This is a bit unfortunate.

The second parameter to `Vararg` is a length, which can also be either unspecified (as above), or specified:

In [46]:
Tuple{Int,Vararg{Int,2}}

Tuple{Int64, Int64, Int64}

## Part 3. Larger type domains

### Union types

A type can be thought of as a set of possible values. A type expresses *uncertainty* about which value we have. You can do set operations on them.

In [47]:
Union{Int64,Float64}

Union{Float64, Int64}

In [48]:
1 isa Union{Int64,Float64}

true

In [49]:
Int64 <: Number

true

In [50]:
Int64 <: Union{Int64,Float64}

true

In [51]:
Union{Int,String} <: Union{Int,String,Float32}

true

In [54]:
typeintersect(Union{Int,String}, Union{Int,String,Float32})

Union{Int64, String}

In [55]:
IntOrString = Union{Int,AbstractString}


Union{Int64, AbstractString}

In [56]:
1 :: IntOrString

1

In [57]:
"Hello!" :: IntOrString

"Hello!"

In [58]:
1.0 :: IntOrString

TypeError: TypeError: in typeassert, expected Union{Int64, AbstractString}, got a value of type Float64

Union types naturally lend themselves to missing data.

In [59]:
data = [1.1, missing, 3.2, missing, 5.7, 0.4]

6-element Vector{Union{Missing, Float64}}:
 1.1
  missing
 3.2
  missing
 5.7
 0.4